In [1]:
import pyTigerGraph as tg
import requests
import json
from pathlib import Path

In [2]:
# On-premise
conn = tg.TigerGraphConnection(host="http://127.0.0.1",graphname="graph_drug_disease", username="tigergraph", password="tigergraph", apiToken="2aa016d747ede9gg6da3drslm98srfoj")

In [5]:
# Cloud
conn = tg.TigerGraphConnection(host="https://ddmp.i.tgcloud.io", graphname="graph_drug_disease")
authToken = conn.getToken("982l37jamamujrg5ing9cvh18mt0o9jn", "100000000000000000")
print(authToken)

('afumc45l3t67ji4n2l7b7ffsdg93144n', 1653042566, '2022-05-20 10:29:26')


In [ ]:
# Installing the following queries if not installed yet.

In [10]:
with open('queries/getDRUGS.gsql', 'r') as file:
    gsql_file = file.read().rstrip()
conn.gsql(gsql_file)
conn.gsql('INSTALL QUERY getDRUGS2')

'Start installing queries, about 1 minute ...\ngetDRUGS2 query: curl -X GET \'https://127.0.0.1:9000/query/graph_drug_disease/getDRUGS2?filename=VALUE\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [11]:
with open('queries/getDISEASES.gsql', 'r') as file:
    gsql_file = file.read().rstrip()
conn.gsql(gsql_file)
conn.gsql('INSTALL QUERY getDISEASES2')

'Start installing queries, about 1 minute ...\ngetDISEASES2 query: curl -X GET \'https://127.0.0.1:9000/query/graph_drug_disease/getDISEASES2?filename=VALUE\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [8]:
with open('queries/tg_common_neighbors.gsql', 'r') as file:
    gsql_file = file.read().rstrip()
conn.gsql(gsql_file)
conn.gsql('INSTALL QUERY tg_common_neighbors')

'Start installing queries, about 1 minute ...\ntg_common_neighbors query: curl -X GET \'https://127.0.0.1:9000/query/graph_drug_disease/tg_common_neighbors?a=VALUE&a.type=VERTEX_TYPE&b=VALUE&b.type=VERTEX_TYPE&e_type=VALUE&[print_res=VALUE]\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [9]:
with open('queries/pattern_for_drugs.gsql', 'r') as file:
    gsql_file = file.read().rstrip()
conn.gsql(gsql_file)
conn.gsql('INSTALL QUERY pattern_for_drugs')

'Start installing queries, about 1 minute ...\npattern_for_drugs query: curl -X GET \'https://127.0.0.1:9000/query/graph_drug_disease/pattern_for_drugs?drug=VALUE\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [12]:
with open('queries/pattern_for_disease.gsql', 'r') as file:
    gsql_file = file.read().rstrip()
conn.gsql(gsql_file)
conn.gsql('INSTALL QUERY pattern_for_disease')

'Start installing queries, about 1 minute ...\npattern_for_disease query: curl -X GET \'https://127.0.0.1:9000/query/graph_drug_disease/pattern_for_disease?disease=VALUE\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [ ]:
# A function to process query results in JSON

In [50]:
def json_results_processing(get_patterns, type_node):
    if(type_node == "drug"):
        Vertex_type = 'VertexSet_2'
        
    if(type_node == "disease"):
        Vertex_type = 'VertexSet_1'
        
    
    VertexList = []
    json_object = json.loads(get_patterns)
    #json_object
    results = json_object['results']
    for elem in results:
        for key, value in elem.items():
            #print(key)
            if(key == Vertex_type):
                #print(value)
                for i in value:
                    VertexList.append(i['v_id'])


            #if(key == '@@FinalEdgeSet_3' or key == '@@FinalEdgeSet_4'):
            #    for i in value:
            #        print(i['to_id'])
    VertexSet = set(VertexList)
    return VertexSet

def getNeighborsVertexSet(drug_or_disease, type_node):
    drug_or_disease = "\""  + drug_or_disease + "\"" 
    if(type_node == "drug"):
        query = 'RUN QUERY pattern_for_drugs('+drug_or_disease+')'
    if(type_node == "disease"):
        query = 'RUN QUERY pattern_for_disease('+drug_or_disease+')'
    get_neighbors = conn.gsql(query)
    #print(get_neighbors)
    VertexSet = json_results_processing(get_neighbors, type_node)
    return VertexSet

def getNodes(type_node):
    if(type_node == "drug"):
        query = 'RUN QUERY getDISEASES()'
        filename = "/home/tigergraph/getDISEASES.txt"
    if(type_node == "disease"):
        query = 'RUN QUERY getDRUGS()'
        filename = "/home/tigergraph/getDRUGS.txt"
        
    #if(os.path.exists(filename)==False):
    conn.gsql(query)
        
    getNodes = []
    with open(filename) as file:
        for line in file: 
            line = line.strip() 
            getNodes.append(line) 

        getNodes = set(getNodes)
    return getNodes

def diff_Vertex(set1, set2):
    diff_set = set1 - set2
    if '' in diff_set:
        diff_set.remove('')
    
    return diff_set

def run_common_neighbors_algo(query):
    
    
    try:
        get_common_neighbors = conn.gsql(query)
        json_object = json.loads(get_common_neighbors)
        results = json_object['results']
        closeness = results[0]['closeness']
    except ValueError:
        print(ValueError, query)
        closeness = 0

    
    return closeness
    
def predictions(drug_or_disease, type_node, diff_set, top_k):
    if "\"" in drug_or_disease:
        drug_or_disease = drug_or_disease.replace("\"", "'")
        print(drug_or_disease)
        drug_or_disease = drug_or_disease.replace("'", "\\'")
    drug_or_disease = "\""+ drug_or_disease +"\""
    list_pred_drug_disease = []
    
    if (type_node == "drug"): # predict disease for a drug
        
        for disease in diff_set:
            disease = "\""+ disease +"\""

            query = 'RUN QUERY tg_common_neighbors (('+drug_or_disease+', "DRUG") , ('+disease+', "DISEASES"),["is_prescripted_for", "gene_disease_association", "drug_gene_association", "is_tested_for"], _)'
            
            results = run_common_neighbors_algo(query)
            list_pred_drug_disease.append((drug_or_disease, disease, results))
        list_pred_drug_disease = sorted(list_pred_drug_disease, key=lambda tup: tup[2], reverse=True)
            
    if (type_node == "disease"): # predict drug for a disease
        
        for drug in diff_set:
                
            drug = "\""+ drug +"\""
            query = 'RUN QUERY tg_common_neighbors (('+drug+', "DRUG") , ('+drug_or_disease+', "DISEASES"),["is_prescripted_for", "gene_disease_association", "drug_gene_association", "is_tested_for"], _)'
            results = run_common_neighbors_algo(query)
            list_pred_drug_disease.append((drug, drug_or_disease, results))
        list_pred_drug_disease = sorted(list_pred_drug_disease, key=lambda tup: tup[2], reverse=True)


    
    return list_pred_drug_disease[:top_k]



In [53]:
# List of diseases used to predict drugs


list_diseases = ["PITUITARY_APOPLEXY", "POLAND_SYNDROME",\
                 "KEPPEN-LUBINSKY_SYNDROME", "ACUTE_PAIN",\
                 "FETAL_DISTRESS", "GONADAL_DISORDERS",\
                 "ANGIOMATOSIS",  "OBESITY", "COLORECTAL_NEOPLASMS"]
list_diseases = ["BRAIN_ISCHEMIA"]

folder = "../results/diseases/"
Path(folder).mkdir(parents=True, exist_ok=True)

for disease_name in list_diseases:
    print(disease_name)
    neighbors_drugs = getNeighborsVertexSet(disease_name, "disease")
    print(neighbors_drugs)
    set_drugs = getNodes("disease")
    print(len(set_drugs))
    diff_drugs = diff_Vertex(set_drugs, neighbors_drugs)
    print(len(diff_drugs))

    results = predictions(disease_name, "disease", diff_drugs, 20)
    folder_disease_name = folder+"/"+disease_name+"/"
    Path(folder_disease_name).mkdir(parents=True, exist_ok=True)
    f = open(folder_disease_name+"predictions"+".txt", "w")
    for res in results:
        #print(res)
        f.write(res[0]+","+str(res[2])+"\n")
    f.close()

BRAIN_ISCHEMIA
{'EDARAVONE', 'TRIFLUSAL', 'DABIGATRAN', 'AMLODIPINE', 'DEFEROXAMINE', 'EPTIFIBATIDE', 'FLUOXETINE', 'NERVE_GROWTH_FACTORS', 'SIROLIMUS', 'VERAPAMIL', 'TRAZODONE', 'ASPIRIN', 'CEREBROLYSIN', 'CANNABIDIOL', 'NIMODIPINE', 'PIOGLITAZONE', 'CLOPIDOGREL', 'LOSARTAN', 'URIC_ACID', 'ANGIOTENSIN-CONVERTING_ENZYME_INHIBITORS', 'CILOSTAZOL', 'RIVAROXABAN', 'WARFARIN'}
59671
59647


In [62]:
# List of drugs for predicting their use in a disease
list_drugs = ["TRAZODONE", "VITAMIN_A", "LIDOCAINE", "CYTOFLAVIN"]

folder = "../results/drugs/"
Path(folder).mkdir(parents=True, exist_ok=True)

for drug_name in list_drugs:

    neighbors_diseases = getNeighborsVertexSet(drug_name, "drug")
    #print(neighbors_drugs)
    set_diseases = getNodes("drug")
    #print(len(set_drugs))
    diff = diff_Vertex(set_diseases, neighbors_diseases)
    #print(len(diff_drugs))

    results = predictions(drug_name, "drug", diff, 20)
    folder_drug_name = folder+"/"+drug_name+"/"
    Path(folder_drug_name).mkdir(parents=True, exist_ok=True)
    f = open(folder_drug_name+"predictions"+".txt", "w")
    for res in results:
        #print(res)
        f.write(res[1]+","+str(res[2])+"\n")
    f.close()

# Parallel version

In [55]:
# get all diseases for computing their predictions
#query = 'RUN QUERY getDISEASES()'
getDISEASES = []
filename = "/home/tigergraph/getDISEASES.txt"
#if(os.path.exists(filename)==False):
#conn.gsql(query)
with open(filename) as file:
    for line in file: 
        line = line.strip() 
        getDISEASES.append(line) 

    getDISEASES = set(getDISEASES)
getDISEASES = list(getDISEASES)
getDISEASES.remove('')
getDISEASES[:10]

['ORAL_ULCER',
 'MINICORE_MYOPATHY_WITH_EXTERNAL_OPHTHALMOPLEGIA',
 'HYPERTHYROXINEMIA',
 'SQUAMOUS_CELL_CARCINOMA_OF_HEAD_AND_NECK',
 'HYPERPITUITARISM',
 'ATHEROSCLEROSIS',
 'SLEEP_WAKE_DISORDERS',
 'TRIGEMINAL_NEURALGIA',
 'HOLT-ORAM_SYNDROME',
 'MATURITY-ONSET_DIABETES_OF_THE_YOUNG']

# Prediction for all diseases

In [ ]:
import time
import math
from multiprocessing import Pool
from multiprocessing import freeze_support
from pathlib import Path

folder = "../results/diseases/"
Path(folder).mkdir(parents=True, exist_ok=True)

getDISEASES = ["BRAIN_ISCHEMIA", "COVID-19", "CROHN_DISEASE", "DENTAL_CARIES"]

'''Define function to run mutiple processors and pool the results together'''
def run_multiprocessing(func, i, n_processors):
    with Pool(processes=n_processors) as pool:
        return pool.map(func, i)

def sequential_prediction_algo(disease_name):
    #print(disease_name)
    neighbors_drugs = getNeighborsVertexSet(disease_name, "disease")
    #print(neighbors_drugs)
    set_drugs = getNodes("disease")
    #print(len(set_drugs))
    diff_drugs = diff_Vertex(set_drugs, neighbors_drugs)
    #print(len(diff_drugs))

    results = predictions(disease_name, "disease", diff_drugs, 20)
    folder_disease_name = folder+"/"+disease_name+"/"
    Path(folder_disease_name).mkdir(parents=True, exist_ok=True)
    f = open(folder_disease_name+"predictions"+".txt", "w")
    for res in results:
        #print(res)
        f.write(res[0]+","+str(res[2])+"\n")
    f.close()
    
start = time.clock()

'''
set up parameters required by the task
'''


n_processors = 5
list_disease = getDISEASES

'''
pass the task function, followed by the parameters to processors
'''
out = run_multiprocessing(sequential_prediction_algo, list_disease, n_processors)

print("Input length: {}".format(len(list_disease)))
print("Output length: {}".format(len(out)))
print("Mutiprocessing time: {}mins\n".format((time.clock()-start)/60))
print("Mutiprocessing time: {}secs\n".format((time.clock()-start)))